<a href="https://colab.research.google.com/github/Romanchenko-RS/ML/blob/main/%D0%93%D0%B8%D0%B1%D1%80%D0%B8%D0%B4%D0%BD%D1%8B%D0%B5_%D1%80%D0%B5%D0%BA%D0%BB%D0%BC%D0%B5%D0%BD%D0%B4%D0%B0%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D1%8B%D0%B5_%D1%81%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D1%8B_%D0%A0%D0%BE%D0%BC%D0%B0%D0%BD%D1%87%D0%B5%D0%BD%D0%BA%D0%BE_%D0%A0_%D0%A1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Установим библиотеку surprise**

In [1]:
pip install numpy pandas scikit-learn surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357252 sha256=14d3df46a2cea1aa0c966eff6e668a095603c471292c32686b5fbf9decba33c2
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


**Импорт необходимых библиотек**

In [36]:
import pandas as pd
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import random

**Получение данных**

In [3]:
!wget 'https://drive.google.com/uc?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx' -O MovieLens.zip

--2024-06-28 09:47:26--  https://drive.google.com/uc?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx
Resolving drive.google.com (drive.google.com)... 74.125.26.113, 74.125.26.138, 74.125.26.102, ...
Connecting to drive.google.com (drive.google.com)|74.125.26.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx [following]
--2024-06-28 09:47:26--  https://drive.usercontent.google.com/download?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 172.217.204.132, 2607:f8b0:400c:c15::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|172.217.204.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 847695 (828K) [application/octet-stream]
Saving to: ‘MovieLens.zip’

MovieLens.zip       100%[===================>] 827.83K  --.-KB/s    in 0.008s  

2024-06-28 09:47:27 (101 MB/s) - ‘M

In [4]:
!unzip MovieLens.zip

Archive:  MovieLens.zip
  inflating: links.csv               
  inflating: movies.csv              
  inflating: ratings.csv             
  inflating: tags.csv                


In [9]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [17]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [18]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [21]:
# Подготовка данных для Surprise
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

In [39]:
# Создание модели SVD
algo = SVD()
trainset = data.build_full_trainset()
algo.fit(trainset)

**Контентная рекомендация**

In [28]:
def get_content_recommendations(movie_id, movies, k=5):
    tfidf = TfidfVectorizer(stop_words='english')
    movies['genres'] = movies['genres'].fillna('')
    tfidf_matrix = tfidf.fit_transform(movies['genres'])

    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

    idx = movies.index[movies['movieId'] == movie_id].tolist()[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:k+1]

    movie_indices = [i[0] for i in sim_scores]
    return movies.iloc[movie_indices].movieId.tolist()

**Коллаборативная рекомендация**

In [29]:
def get_collaborative_recommendations(user_id, algo, movies, k=5):
    movie_ids = movies['movieId'].unique()
    predictions = [(movie_id, algo.predict(user_id, movie_id).est) for movie_id in movie_ids]
    predictions = sorted(predictions, key=lambda x: x[1], reverse=True)

    recommendations = [pred[0] for pred in predictions[:k]]
    return recommendations

**Гибридная система рекомендаций**

In [33]:
def hybrid_recommendations(user_id, movie_id, ratings, movies, algo, k=5, n_candidates=10):
    content_recs = get_content_recommendations(movie_id, movies, n_candidates)
    collaborative_recs = get_collaborative_recommendations(user_id, algo, movies, n_candidates)

    hybrid_recs = list(set(content_recs + collaborative_recs))

    # Случайная выборка k рекомендаций из объединенного списка
    if len(hybrid_recs) > k:
        hybrid_recs = random.sample(hybrid_recs, k)

    return movies[movies['movieId'].isin(hybrid_recs)].title.tolist()

**Результат**

In [40]:
user_id = 1
movie_id = 1

recommendations = hybrid_recommendations(user_id, movie_id, ratings, movies, algo, k=5)
print(f"Рекомендации для пользователя {user_id} на основе фильма ID {movie_id}:")
recommendations

Рекомендации для пользователя 1 на основе фильма ID 1:


['Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)',
 'Citizen Kane (1941)',
 'Adventures of Rocky and Bullwinkle, The (2000)',
 'Monsters, Inc. (2001)',
 'Tale of Despereaux, The (2008)']

**Вывод**

В результате объединения контентной и коллаборативной рекомендательных систем, получили одну единую гибридную рекомендательную систему, которая каждый раз отдаёт 5 наиболее подходящих и релевантных фильма под конкретного пользователя и конкретный фильм.